## Web API's continued: Face Recognition, Entity Extraction, etc

Now let's try to play with a few APIs that are a bit more complex than the ones that we dealt earlier.

## FacePlusPlus API: Face Recognition

Let's start with the FacePlusPlus API that allows us to recognize faces. We will call the API through Mashape, which will also allow us to learn about _headers_, which is an additional piece of information that we send to APIs, in addition to parameters. The documentation of Face++ on Mashape can be found at https://market.mashape.com/faceplusplus/faceplusplus-face-detection.

We will start by analyzing the image below, which is accessible through this URL: http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg

![Image from NY Times](http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg)

In [ ]:
import requests
import json

facepp_url = "https://faceplusplus-faceplusplus.p.mashape.com/detection/detect"
img_url = "http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg"

headers = {
  "X-Mashape-Key": "zG3wec50exmshxNoF1NMHNRH37GYp1d7oW8jsnWwIMTeMmALxg",
  "Accept": "application/json"
}
parameters = {
    'attributes': 'glass,pose,gender,age,race,smiling',
    'url': img_url
}

data = requests.get(facepp_url, params=parameters, headers=headers, verify=True).json()


In [ ]:
data

In [ ]:
data.keys()

In [ ]:
# The "face" attribute contains a list, and each element of the list is a dictionary
len(data["face"])

#### Exercise

* Print the gender, age, race, and smiling attributes for each face
* Do an image search and get an image URL from the Internet, preferably with multiple faces. Repeat the task above for the new image.

In [ ]:
# your code here

### Interacting with the IBM Watson Alchemy API; POST vs GET

Another useful API, especially when dealing with text, is the IBM Watson [Alchemy Language API](https://www.ibm.com/watson/developercloud/alchemy-language.html), which offers a variety of text analysis functionalities, such as sentiment analysis, entity extraction, keyword extraction, etc (see the main page for tutorials and the [API documentation](https://www.ibm.com/watson/developercloud/alchemy-language/api/v1/) for detailed description of each API call).

#### TextGetTextSentiment call

Full Documentation of the call at https://www.ibm.com/watson/developercloud/alchemy-language/api/v1/#sentiment

We will first start with the TextGetTextSentiment API call, which takes as input a piece of text, and returns a sentiment value associated with it. 

With this API call, we will use for the first time the "POST" method, as opposed to the "GET" method that we used so far. The "POST" method is used when the size of the parameters that we need to pass to the API is to big (e.g., analyzing a piece of long text for sentiment). In such cases, we need to use the "POST" options as opposed to the "GET" function (although admittedly, many APIs will try to be forgiving).

The call below gets as input a "text" variable, and returns back the sentiment of the text.

In [ ]:
import requests
import json

url = "http://access.alchemyapi.com/calls/text/TextGetTextSentiment"

# You can register and get your own key
api_key = '4b46c7859a7be311b6f9389b12504e302cac0a55'

text = '''
If things go as promised, Trump won’t be there Thursday when Fox hosts the final Republican debate before Monday’s Iowa presidential caucuses. He says he’s backing out because of a taunting statement from Fox, though his detractors accuse him of dodging a last showdown with his chief rival, Sen. Ted Cruz (Tex.). Instead, Trump has made plans to materialize elsewhere in Iowa, hosting a benefit for wounded veterans — counter-programming on a ­Trumpian scale of swagger.

His threatened absence from the debate stage is a demonstration of Trump’s perception of his own self-worth, his verifiable status as a ratings-generating gargantuan whose screen persona can translate into millions of advertising dollars. In a sense, it’s an act of subversion by a candidate who has broken all the normal rules of modern campaigns. But it’s also a manifestation of Trump’s philosophy about getting what he wants when he wants it.
'''

headers = {
  "Accept": "application/json"
}

parameters = {
    'outputMode': 'json',
    'apikey' : api_key,
    'text': text,
}

resp = requests.post(url, params=parameters, headers=headers)
data = json.loads(resp.text)

data

#### URLGetRankedNamedEntities call

Full Documentation of the call at https://www.ibm.com/watson/developercloud/alchemy-language/api/v1/#entities


And here is a different API call that extracts entities from the text, and also the sentiment for each of these entities. You will also see that there is the capability of "normalizing" each entity, so that two different ways of saying the same thing get mapped to the same entity. So for example, "President Trump" and "Donald Trump" get mapped to the same Knowledge Graph entity.

In [ ]:
import requests

url = "http://gateway-a.watsonplatform.net/calls/url/URLGetRankedNamedEntities"

# You can register and get your own key
api_key = '4b46c7859a7be311b6f9389b12504e302cac0a55'

text_url = 'https://www.washingtonpost.com/politics/question-whats-clintons-message-answer-all-of-the-above/2016/02/06/5df28fcc-cc4a-11e5-88ff-e2d1b4289c2f_story.html'
headers = {
  "Accept": "application/json"
}

parameters = {
    'outputMode': 'json',
    'apikey' : api_key,
    'sentiment' :1,
    'knowledgeGraph': 1,
    'url': text_url
}

resp = requests.post(url, params=parameters, headers=headers)
data = resp.json()


In [ ]:
# Let's see what we get back as top-level attributes
data.keys()

In [ ]:
# Let' see the entities list
data["entities"]

In [ ]:
# Let' see the first entity. Notice the "disambiguated" attribute that
# points to "canonical" versions of the entity, in DBPedia, Freebase, OpenCYC, YAGO, etc
data["entities"][0]

#### Exercise

* Fetch the main page of NY Times. Print the entities that are currently being discussed in the news, together with their relevance value and the associated sentiment.
* _Optional:_ Use the NY Times API to fetch the Top Stories News. You can register and get an API key at https://developer.nytimes.com/. The `Top Stories V2 API` provides the details of the news of the day: (The API call documentation is at https://developer.nytimes.com/top_stories_v2.json and the API Call is  https://api.nytimes.com/svc/topstories/v2/home.json?api-key=PUTYOURKEYHERE). Repeat the entity extraction process from above.

In [ ]:
# your code here

### Exercise: Using the Spotify API

We will now use the Spotify API to get information about an artist. The documentation of the calls is at https://developer.spotify.com/web-api/endpoint-reference/. For now, use only the calls that do not require an OAuth authentication. 

Tasks:
* We can first find the id of an artist using the `/v1/search?type=artist` API call. The documentation of the `search-item` endpoint is at https://developer.spotify.com/web-api/search-item/.
* Once you get back the ID of the artist, use the `get artist` endpoint, to get further information about the artist: https://developer.spotify.com/web-api/get-artist/
* Study the documentation and figure out how to get the albums of an article, the top tracks for an artist, and the related artists.



